# building an autonomous research agent
- rag with llamaindex
- 1. routing: add decision making to route requests to multiple tools

- 2. tool use: create an interface for the agents to select a tool and generate the right argument for that tool

- 3. multi-step reasoning with a range of tool using the LLM while maintaining language throughout that process

- 4. let users optionally inject guidance during intermediate steps (during reasoning)

In [1]:
import sys
import os
import nest_asyncio # make jupyter nb play nicelly with llamaindex
from helper import load_api_key
OPENAI_API_KEY = load_api_key('openai_pat.txt')
OPENAI_ORG_KEY = load_api_key('openai_org.txt')

In [6]:
import openai 
from openai import OpenAI

# Set up the OpenAI client
client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_KEY
)

## Rounting

In [ ]:
# load data
from llama_index.core import SimpleDirectoryReader

# read pdf into a parsed documentation
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

### Define LLM and Embedding model

In [ ]:
# split the documents into even sized chucks
# split on the order of sentences
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
# define documents into notes 
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
# global confile setting 
# specify the LLM embedding model to use 
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from utils import *

Settings.llm = OpenAI(model="gpt-4o") 
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")